In [2]:
##### importing custom modules from the projects folder
import sys
from pathlib import Path
# Add project root to sys.path
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.append(str(PROJECT_ROOT))
import modules.scrapers as scrapers
##### -------------------------------------------------
from datetime import date

##### PARAMS ######
league = 'nfl'
export = True
today = date.today()
season = 2025
week = int(input("What week is it >>>>>> ? "))
print(today, "Week:", week)

scraper = scrapers.scrapers(
    season = season,
    week = week,
    today = today
)

2025-08-16 Week: 0


In [ ]:
scraper.cbs_projections(export = export)

In [ ]:
# doesn't work anymore
scraper.cbs_rankings(export = export)

In [ ]:
scraper.cbs_game_scores(export = export)

In [ ]:
scraper.ffp_ecr_rankings(export = export)

In [ ]:
scraper.ffp_rankings(export = export)

In [ ]:
scraper.espn_projections(export = export)

In [ ]:
scraper.espn_rankings(export = export)

In [ ]:
scraper.nfl_projections(export = export)

In [ ]:
scraper.nfl_rankings(export = export)

In [ ]:
scraper.bp_lines(export = export)

#  ADP

In [ ]:
scraper.cbs_adp(
    export = export,
    cbs_adp_url = "https://www.cbssports.com/fantasy/football/draft/averages/"
)
scraper.scraped_dfs['adps']['cbs']

C:\projection\offseason\cbs_adp_2025-00_2025-08-16.csv


,outlet,date,playerId,name,shortName,pos,team,adp,high,low
0,cbs,2025-08-16,2966320,Ja'Marr Chase,J Chase,WR,CIN,2.12,1,7
1,cbs,2025-08-16,3168163,Bijan Robinson,B Robinson,RB,ATL,2.28,1,5
2,cbs,2025-08-16,2185957,Saquon Barkley,S Barkley,RB,PHI,2.34,1,7
3,cbs,2025-08-16,3162723,Jahmyr Gibbs,J Gibbs,RB,DET,4.78,3,7
4,cbs,2025-08-16,2061188,Derrick Henry,D Henry,RB,BAL,5.69,3,9
...,...,...,...,...,...,...,...,...,...,...
205,cbs,2025-08-16,2131148,Samaje Perine,S Perine,RB,CIN,160.90,128,202
206,cbs,2025-08-16,2968527,Jake Moody,J Moody,K,SF,161.38,86,189
207,cbs,2025-08-16,2866214,Calvin Austin III,C Austin III,WR,PIT,163.11,140,191
208,cbs,2025-08-16,3121084,Will Reichard,W Reichard,K,MIN,168.00,132,192


In [ ]:
scraper.ffp_adp(
        export = export,
        fp_adp_url = r"https://www.fantasypros.com/nfl/adp/half-point-ppr-overall.php",
        database_table = 'adp'
    )
scraper.scraped_dfs['adps']['ffp']

,outlet,date,playerId,name,shortName,pos,team,adp,high,low
0,yahoo,2025-08-16,19788,Ja'Marr Chase,NaN,WR,CIN,1,NaN,NaN
1,fantrax,2025-08-16,19788,Ja'Marr Chase,NaN,WR,CIN,1,NaN,NaN
2,ffc,2025-08-16,19788,Ja'Marr Chase,NaN,WR,CIN,1,NaN,NaN
3,sleeper,2025-08-16,19788,Ja'Marr Chase,NaN,WR,CIN,1.0,NaN,NaN
4,yahoo,2025-08-16,17240,Saquon Barkley,NaN,RB,PHI,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1359,sleeper,2025-08-16,25977,Isaiah Davis,NaN,RB,NYJ,329.0,NaN,NaN
1360,yahoo,2025-08-16,27454,Jaylin Lane,NaN,WR,WAS,,NaN,NaN
1361,fantrax,2025-08-16,27454,Jaylin Lane,NaN,WR,WAS,,NaN,NaN
1362,ffc,2025-08-16,27454,Jaylin Lane,NaN,WR,WAS,330,NaN,NaN


In [ ]:
scraper.espn_adp(
        export = export,
        espn_adp_url = r"https://fantasy.espn.com/football/livedraftresults",
        database_table = 'adp',
        date_override = None
    )
scraper.scraped_dfs['adps']['espn']

<string>:8: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 8 of the file c:\Users\jrbrz\Desktop\projects\projects\fantasyfootball\modules\scrapers.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

<string>:32: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 32 of the file c:\Users\jrbrz\Desktop\projects\projects\fantasyfootball\modules\scraper

,outlet,date,playerId,name,shortName,pos,team,adp,high,low
0,espn,2025-08-16,4362628,Ja'Marr Chase,NaN,WR,Cin,1.6,NaN,NaN
1,espn,2025-08-16,4430807,Bijan Robinson,NaN,RB,Atl,3.1,NaN,NaN
2,espn,2025-08-16,3929630,Saquon Barkley,NaN,RB,Phi,3.6,NaN,NaN
3,espn,2025-08-16,4262921,Justin Jefferson,NaN,WR,Min,4.7,NaN,NaN
4,espn,2025-08-16,4429795,Jahmyr Gibbs,NaN,RB,Det,5.4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
495,espn,2025-08-16,3931391,Trenton Irwin,NaN,WR,Jax,170.0,NaN,NaN
496,espn,2025-08-16,3040569,Trent Taylor,NaN,WR,SF,170.0,NaN,NaN
497,espn,2025-08-16,4429086,Michael Mayer,NaN,TE,LV,170.0,NaN,NaN
498,espn,2025-08-16,15091,Randy Bullock,NaN,K,FA,170.0,NaN,NaN
